In [3]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("../input/")
import d2l_jay9460 as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float32
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

## GRU

$$
R_{t} = σ(X_tW_{xr} + H_{t−1}W_{hr} + b_r)\\    
Z_{t} = σ(X_tW_{xz} + H_{t−1}W_{hz} + b_z)\\  
\widetilde{H}_t = tanh(X_tW_{xh} + (R_t ⊙H_{t−1})W_{hh} + b_h)\\
H_t = Z_t⊙H_{t−1} + (1−Z_t)⊙\widetilde{H}_t
$$

In [4]:
class GRU():
    def __init__(self, num_inputs, num_hiddens, num_outputs):
        self.num_inputs = num_inputs
        self.num_hiddens = num_hiddens
        self.num_outputs = num_outputs
        
    def get_params(self):
        def _set_W(shape):
            W = torch.tensor(np.random.normal(0, 0.01, size=shape), device = device, dtype = dtype)
            return nn.Parameter(W, requires_grad=True)
            
        def set_param():
            W_x = _set_W(shape = (self.num_inputs, self.num_hiddens))
            W_h = _set_W(shape = (self.num_hiddens, self.num_hiddens))
            b   = nn.Parameter(torch.zeros(self.num_hiddens, device=device, dtype=dtype), requires_grad=True)
            return (W_x, W_h, b)
        
        W_xr, W_hr, b_r = set_param()
        W_xz, W_hz, b_z = set_param()
        W_xh, W_hh, b_h = set_param()
        
        W_ho = _set_W(shape = (self.num_hiddens, self.num_outputs))
        b_o  = nn.Parameter(torch.zeros(self.num_outputs, device=device, dtype=dtype), requires_grad=True)
        return nn.ParameterList([W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_ho, b_o])
    
    def init_state(self, batch_size, num_hiddens, device=device):
        return torch.zeros((batch_size, num_hiddens), device=device, dtype=dtype)
        
    def gru(self, inputs, state, params):
        outputs = []
        W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_ho, b_o = params
        H = state
        for X in inputs:
            R = torch.sigmoid( torch.matmul(X, W_xr) + torch.matmul(H, W_hr) + b_r )
            Z = torch.sigmoid( torch.matmul(X, W_xz) + torch.matmul(H, W_hz) + b_z )
            H_candidate = torch.tanh( torch.matmul(X, W_xh) + R * torch.matmul(H, W_hh) + b_h )
            H = Z * H + (1 - Z) * H_candidate
            Y = torch.matmul(H, W_ho) + b_o
            outputs.append(Y)
        return outputs, H

In [5]:
num_hiddens = 256
gruer = GRU(vocab_size, num_hiddens, vocab_size)
params, state = gruer.get_params(), gruer.init_state(1, num_hiddens)
# print(params,state.shape)

gru = gruer.gru
inputs = d2l.to_onehot(torch.tensor([[char_to_idx['分']]]), n_class = vocab_size)
outputs, state = gru(inputs, state, params)

print(inputs[0].shape, outputs[0].shape, state.shape)

torch.Size([1, 1027]) torch.Size([1, 1027]) torch.Size([1, 256])


In [51]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

d2l.train_and_predict_rnn(gru, gruer.get_params, gruer.init_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, True, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 149.774903, time 1.18 sec
 - 分开 我不要 我不要你我想想你的可爱女人 坏坏的让我 爱爱人 我不要 我不要你我想想你的可爱女人 坏坏的
 - 不分开 我不要 我不要你我想想你的可爱女人 坏坏的让我 爱爱人 我不要 我不要你我想想你的可爱女人 坏坏的
epoch 80, perplexity 32.686437, time 1.18 sec
 - 分开 我不能这样 你知不觉 你已经 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再
 - 不分开 爱你的美女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的
epoch 120, perplexity 6.305357, time 1.20 sec
 - 分开我 不知不觉 我跟了这节奏 后知后觉 我该好好生活 我知不觉 你已经离开我 不知不觉 我跟了这节奏 
 - 不分开 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 我该好好生活 我知不觉 你已经离开我 不
epoch 160, perplexity 2.040157, time 1.16 sec
 - 分开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 不知不觉 
 - 不分开 爱知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我


In [35]:
#简洁实现

num_hiddens=256
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

lr = 1e-2 # 注意调整学习率
gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.024406, time 0.79 sec
 - 分开始乡相信命运 感谢地心引力 让我碰到你 漂亮的让我面红的可爱女人 温柔的让我心疼的可爱女人 透明的让
 - 不分开球我满腔的怒火 我想揍你已经很久 别想躲 说你眼睛看着我 别发抖 快给我抬起头 有话去对医药箱说 别
epoch 80, perplexity 1.011810, time 0.82 sec
 - 分开始共渡每一天 手牵手 一步两步三步四步望著天 看星星 一颗两颗三颗四颗 连成线背著背默默许下心愿 看
 - 不分开 没有你以  我想大声宣布 对你依依不舍 连隔壁邻居都猜到我现在的感受 河边的风 在吹着头发飘动 牵
epoch 120, perplexity 1.015511, time 0.89 sec
 - 分开球我爱你看棒球 想这样没担忧 唱着歌 一直走 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害
 - 不分开 陷入了危险边缘Baby  我的世界已狂风暴雨 Wu  爱情来的太快就像龙卷风 离不开暴风圈来不及逃
epoch 160, perplexity 1.010384, time 0.96 sec
 - 分开 我爱你看棒球 想这样没担忧 唱着歌 一直走 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害
 - 不分开 你爱我 开不了口 周杰伦 才离开没多久就开始 担心今天的你过得好不好 整个画面是你 想你想的睡不著


## LSTM

$$
I_t = σ(X_tW_{xi} + H_{t−1}W_{hi} + b_i) \\
F_t = σ(X_tW_{xf} + H_{t−1}W_{hf} + b_f)\\
O_t = σ(X_tW_{xo} + H_{t−1}W_{ho} + b_o)\\
\widetilde{C}_t = tanh(X_tW_{xc} + H_{t−1}W_{hc} + b_c)\\
C_t = F_t ⊙C_{t−1} + I_t ⊙\widetilde{C}_t\\
H_t = O_t⊙tanh(C_t)
$$

In [29]:
class LSTM():
    def __init__(self, num_inputs, num_hiddens, num_outputs):
        self.num_inputs = num_inputs
        self.num_hiddens = num_hiddens
        self.num_outputs = num_outputs
    
    def get_params(self):
        def _set_W(shape):
            W = torch.tensor(np.random.normal(0, 0.01, size=shape), device = device, dtype = dtype)
            return nn.Parameter(W, requires_grad=True)
            
        def set_param():
            W_x = _set_W(shape = (self.num_inputs, self.num_hiddens))
            W_h = _set_W(shape = (self.num_hiddens, self.num_hiddens))
            b   = nn.Parameter(torch.zeros(self.num_hiddens, device=device, dtype=dtype), requires_grad=True)
            return (W_x, W_h, b)
        
        W_xi, W_hi, b_i = set_param()
        W_xf, W_hf, b_f = set_param()
        W_xo, W_ho, b_o = set_param()
        W_xc, W_hc, b_c = set_param()
        
        W_hq = _set_W(shape = (self.num_hiddens, self.num_outputs) )
        b_q  = nn.Parameter(torch.zeros(self.num_outputs, device=device, dtype=dtype), requires_grad=True)
        return nn.ParameterList([W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q])
    
    def lstm(self, inputs, state, params):
        W_xi, W_hi, b_i, W_xf, W_hf, b_f, W_xo, W_ho, b_o, W_xc, W_hc, b_c, W_hq, b_q = params
        H, C = state
        outputs = []
        for X in inputs:
            I = torch.sigmoid( torch.matmul(X, W_xi) + torch.matmul(H, W_hi) + b_i )
            F = torch.sigmoid( torch.matmul(X, W_xf) + torch.matmul(H, W_hf) + b_f )
            O = torch.sigmoid( torch.matmul(X, W_xo) + torch.matmul(H, W_ho) + b_o )
            C_candidate = torch.tanh( torch.matmul(X, W_xc) + torch.matmul(H, W_hc) + b_c )
            C = F * C + I * C_candidate
            H = O * torch.tanh(C)
            Y = torch.matmul(H, W_hq) + b_q
            outputs.append(Y)
        return outputs, (H, C)
    
    def init_state(self, batch_size, num_outputs, device = device):
        H = torch.zeros((batch_size, num_outputs), dtype=dtype, device=device)
        C = torch.zeros((batch_size, num_outputs), dtype=dtype, device=device)
        return (H, C)

In [31]:
num_hiddens = 256
lstmer = LSTM(vocab_size, num_hiddens, vocab_size)
params, state = lstmer.get_params(), lstmer.init_state(1, num_hiddens)
# print(params,state[0].shape)

lstm = lstmer.lstm
inputs = d2l.to_onehot(torch.tensor([[char_to_idx['分']]]), n_class = vocab_size)
outputs, state = lstm(inputs, state, params)

print(inputs[0].shape, outputs[0].shape, state[0].shape)

torch.Size([1, 1027]) torch.Size([1, 1027]) torch.Size([1, 256])


In [33]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

d2l.train_and_predict_rnn(lstm, lstmer.get_params, lstmer.init_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, True, num_epochs, num_steps, lr,
                          clipping_theta, batch_size, pred_period, pred_len,
                          prefixes)

epoch 40, perplexity 204.493270, time 1.28 sec
 - 分开 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我
 - 不分开 我不的我 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 我不的 
epoch 80, perplexity 75.061221, time 1.29 sec
 - 分开 我想 你你的爱你 你知  你不 我不要 我不要 我不要 你不 我不要 你不 我不要 我不要 你不 
 - 不分开 我想 你你的爱你 你知  你不 我不要 我不要 我不要 你不 我不要 你不 我不要 我不要 你不 
epoch 120, perplexity 22.797335, time 1.28 sec
 - 分开 我爱你的爱笑 就知的美 快你的美 在小村  什么我 有有我 有你么 我不要 我不要 我不好 是你的
 - 不分开 我不了这样 我知了这样 我知了这生 我的让我 你你的让我面红的可爱女人 坏柔的让我心疼的可爱女人 
epoch 160, perplexity 7.384752, time 1.29 sec
 - 分开 我不要再想 我不 我不 我不要 爱情走的太快就像龙卷风 不能承受我已无处可可躲 我不要再想 我不能
 - 不分开 我知不能 你情的让我爱红 可爱女人 经已已可可简 后以透透我 是你的美我 说伤寄美 如果什么我出的


In [36]:
#简洁实现

num_hiddens=256
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

lr = 1e-2 # 注意调整学习率
lstm_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = d2l.RNNModel(lstm_layer, vocab_size)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.022268, time 1.06 sec
 - 分开 我不能看棒球 想这样没担忧 唱着歌 一直走 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害
 - 不分开 我不能回到你 我不能再想 我不能再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处
epoch 80, perplexity 1.029295, time 1.11 sec
 - 分开始打呼 管家是一只会说法语举止优雅的猪 吸血前会念约翰福音做为弥补 拥有一双蓝色眼睛的凯萨琳公主 专
 - 不分开 我不能回到你身边 我给你的爱写在西元前 深埋在美索不达米亚平原 几十个世纪后出土发现 泥板上的字迹
epoch 120, perplexity 1.011268, time 1.02 sec
 - 分开始打呼 管家是一只会说法语举止优雅的猪 吸血前会念约翰福音做为弥补 拥有一双蓝色眼睛的凯萨琳公主 专
 - 不分开 我不能回到 我不 我不 我不要再想你 爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我不能再想 我
epoch 160, perplexity 1.008309, time 1.03 sec
 - 分开始打呼 管家是一只会说法语举止优雅的猪 吸血前会念约翰福音做为弥补 拥有一双蓝色眼睛的凯萨琳公主 专
 - 不分开 爱能不能够永远单纯没有悲哀 我 想带你骑单车 我 想和你看棒球 想这样没担忧 唱着歌 一直走 我想


## Deep RNN

$$
\boldsymbol{H}_t^{(1)} = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(1)} + \boldsymbol{H}_{t-1}^{(1)} \boldsymbol{W}_{hh}^{(1)} + \boldsymbol{b}_h^{(1)})\\
\boldsymbol{H}_t^{(\ell)} = \phi(\boldsymbol{H}_t^{(\ell-1)} \boldsymbol{W}_{xh}^{(\ell)} + \boldsymbol{H}_{t-1}^{(\ell)} \boldsymbol{W}_{hh}^{(\ell)} + \boldsymbol{b}_h^{(\ell)})\\
\boldsymbol{O}_t = \boldsymbol{H}_t^{(L)} \boldsymbol{W}_{hq} + \boldsymbol{b}_q
$$

In [37]:
num_hiddens=256
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

lr = 1e-2 # 注意调整学习率

gru_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens,num_layers=2)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)

epoch 40, perplexity 1.351494, time 1.46 sec
 - 分开 这故事 告诉我 别怪我 说你怎么面对我 别怪我 说你怎么面对我 别怪我 说你怎么面对我 别怪我 说
 - 不分开却已了伊斯坦堡 就只想你开 我不  那处悲剧 我想就这样牵着你的手不放开 爱可不可以简简单单没有伤害
epoch 80, perplexity 1.014319, time 1.49 sec
 - 分开都妈出看 温柔的让我心疼的可爱女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯
 - 不分开不要再这样打我妈妈 难道你手不会痛吗 不要再这样打我妈妈 难道你手不会痛吗 其实我回家就想要阻止一切
epoch 120, perplexity 1.013718, time 1.47 sec
 - 分开的直走 我想很伊斯坦堡 就像是童话故事  有教堂有城堡 每天忙碌地的寻找 到底什么我想要 却发现迷了
 - 不分开不要再这样打我妈妈 难道你手不会痛吗 不要再这样打我妈妈 难道你手不会痛吗 我叫你爸 你打我妈 这样
epoch 160, perplexity 1.012212, time 1.47 sec
 - 分开已经发出痛 有什么不妥 有话就直说 别窝在角落 不爽就反驳 到底拽什么 懂不懂篮球 有种不要走 三对
 - 不分开不要再这样打我妈妈 难道你手不会痛吗 不要再这样打我妈妈 难道你手不会痛吗 我叫你爸 你打我妈 这样


## Bidirectional RNN


$$ 
\begin{aligned} \overrightarrow{\boldsymbol{H}}_t &= \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(f)} + \overrightarrow{\boldsymbol{H}}_{t-1} \boldsymbol{W}_{hh}^{(f)} + \boldsymbol{b}_h^{(f)})\\
\overleftarrow{\boldsymbol{H}}_t &= \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh}^{(b)} + \overleftarrow{\boldsymbol{H}}_{t+1} \boldsymbol{W}_{hh}^{(b)} + \boldsymbol{b}_h^{(b)}) \end{aligned} $$
$$
\boldsymbol{H}_t=(\overrightarrow{\boldsymbol{H}}_{t}, \overleftarrow{\boldsymbol{H}}_t)
$$
$$
\boldsymbol{O}_t = \boldsymbol{H}_t \boldsymbol{W}_{hq} + \boldsymbol{b}_q
$$

In [ ]:
num_hiddens=128
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e-2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

lr = 1e-2 # 注意调整学习率

gru_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens,bidirectional=True)
model = d2l.RNNModel(gru_layer, vocab_size).to(device)
d2l.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes)